<a href="https://colab.research.google.com/github/samarasrs/ELE-32/blob/main/ELE32_Lab03_Codificacao_convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from random import choices, getrandbits
import numpy as np

In [ ]:
#CanalBSC

class CanalBSC:
    def __init__(self, entrada, probErro):
        self.entrada = entrada
        self.saida = []
        self.probErro = probErro

    def transmitir(self):
        acertou = [True, False]
        prob = [1 - self.probErro, self.probErro]
        
        for j in self.entrada:
          aux = choices(acertou, weights=prob, k=1)
          if aux[0]:
            self.saida.append(j)
          else:
            if j == 1:
                self.saida.append(0)
            else:
                self.saida.append(1)
          
            
        return self.saida

In [ ]:
def Int2Bin(numero, quant_bits):
  B = []
  num = numero
  while num > 0:
    B.insert(0, num%2)
    num = int(num/2)

  while len(B) < quant_bits:
    B.insert(0, 0)

  return B


def Bin2Int(lista_num):
  exp = len(lista_num) - 1
  num = 0
  for i in lista_num:
    num = num + i*2**exp
    exp = exp - 1

  return num


In [ ]:
#Codificador

class Codificador:
  def __init__(self, quant_memoria, pol_gerador):
    self.quant_memoria = quant_memoria
    self.pol_gerador = pol_gerador
    self.memoria = [0]*(self.quant_memoria) #memoria se inicia com zeros
    
  def cod(self, memoria_atual, bit_entrada):
    saida_t = []
    memoria = memoria_atual
    
    for pol in self.pol_gerador:
      saida = bit_entrada
      for k in range(len(pol[1:])):
        if pol[k+1]:
          saida = saida + memoria[k]
      saida = saida % 2
      saida_t.append(int(saida))
    
    memoria = memoria[:-1]
    memoria.insert(0,bit_entrada)
    
    self.memoria = memoria
    return saida_t

  def get_memoria(self):
    return self.memoria

In [ ]:
#Decodificador

class Decodificador:
  def __init__(self, recebido, quant_memoria, pol_gerador):
    self.recebido = recebido
    self.quant_memoria = quant_memoria
    self.quant_estados = 2**quant_memoria
    self.peso_acumulado = [0]*(self.quant_estados)
    self.caminhos = []
    self.codificador = Codificador(quant_memoria,pol_gerador)

  def calcular_custo(self, recebido, estimado):
    custo = (np.array(recebido) + np.array(estimado))%2
    custo = np.sum(custo)
    return custo

  def decod_completo(self):
    anterior = [None]*(self.quant_estados)
    anterior[0] = 0
    for item in self.recebido:
      anterior = self.decod(anterior, item)
      self.caminhos.append(anterior)
    
    idx = 0
    
    for i in range(self.quant_estados):
      if self.peso_acumulado[i] <  self.peso_acumulado[idx]:
        idx = i
      elif self.peso_acumulado[i] <  self.peso_acumulado[idx]:
        trocar = getrandbits(1)
        if trocar:
          idx = i


    self.caminhos.reverse()
    estimada = []

    aux = 2**(self.quant_memoria - 1)
    for item in self.caminhos:
      if idx >= aux:
        estimada.insert(0,1)
      else:
        estimada.insert(0,0)
      idx = item[idx]


    return estimada


  def decod(self, anterior, recebido):
    atual = [None]*(self.quant_estados)
        
    pesos = [0]*(self.quant_estados)

    for i in range(self.quant_estados):
      
      if anterior[i] != None:
        for j in range(2):
          memoria = Int2Bin(i, self.quant_memoria)

          codificado = self.codificador.cod(memoria, j)
          memoria = self.codificador.get_memoria()
          
          custo = self.calcular_custo(recebido, codificado)
          
          idx = Bin2Int(memoria)
          
          if atual[idx] == None:
            atual[idx] = i
            pesos[idx] = self.peso_acumulado[i] + custo
            
          elif pesos[idx] > (self.peso_acumulado[i] + custo):
            atual[idx] = i
            pesos[idx] = self.peso_acumulado[i] + custo

          elif pesos[idx] == (self.peso_acumulado[i] + custo):
            trocar = getrandbits(1)
            if trocar:
              atual[idx] = i
              pesos[idx] = self.peso_acumulado[i] + custo

    self.peso_acumulado = pesos
        
    return atual

In [ ]:
REP = 10000
#PROB_P = [0.5,0.2, 0.1, 0.05, 0.02, 0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001, 0.000005, 0.000002,0.000001]
PROB_P = [0.328641,0.294804,0.216440,0.132633,0.061033,0.017837,0.002539,0.000116] #prob para o comparativo

pol_3 = [[1, 0, 1, 1], [1, 1, 0, 1], [1, 1, 1, 1]]
pol_4 = [[1, 0, 1, 0, 1], [1, 1, 0, 1, 1], [1, 1, 1, 1, 1]]
pol_6 = [[1, 0, 0, 1, 1, 1, 1], [1, 0, 1, 0, 1, 1, 1], [1, 1, 0, 1, 1, 0, 1]]

codi3 = Codificador(3, pol_3)
codi4 = Codificador(4, pol_4)
codi6 = Codificador(6, pol_6)

Pb3 = []
Pb4 = []
Pb6 = []


for j in PROB_P:
  print(j)

  enviado = []

  recebida3 = []
  recebida4 = []
  recebida6 = []

  estimada3 = []
  estimada4 = []
  estimada6 = []

  for i in range(REP):
    bit = getrandbits(1)
    enviado.append(bit)

    transmitida3 = codi3.cod(codi3.get_memoria(), bit)
    recebida3.append(CanalBSC(transmitida3,j).transmitir())

    transmitida4 = codi4.cod(codi4.get_memoria(), bit)
    recebida4.append(CanalBSC(transmitida4,j).transmitir())
   
    transmitida6 = codi6.cod(codi6.get_memoria(), bit)
    recebida6.append(CanalBSC(transmitida6,j).transmitir())
   
  estimada3 = Decodificador(recebida3, 3, pol_3).decod_completo()
  estimada4 = Decodificador(recebida4, 4, pol_4).decod_completo()
  estimada6 = Decodificador(recebida6, 6, pol_6).decod_completo()
  

  erro3 = np.sum((np.array(estimada3)+np.array(enviado))%2)
  erro4 = np.sum((np.array(estimada4)+np.array(enviado))%2)
  erro6 = np.sum((np.array(estimada6)+np.array(enviado))%2)

  Pb3.append(erro3/REP)
  Pb4.append(erro4/REP)
  Pb6.append(erro6/REP)

print(Pb3)
print(Pb4)
print(Pb6)
